In [44]:
import re
import pandas as pd
import numpy as np
from collections import Counter 
from konlpy.tag import Okt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model

In [45]:
# 기존 X_train csv 파일 전환 후 로드시 X_train.csv 파일로 틀어짐 전처리 
X_train = pd.read_csv('C://users/crid2/nl_test/X_train.csv')
X_train = X_train.drop('Unnamed: 0' , axis=1)
X_train2 = X_train["tokenized"].squeeze()
X_train3 = X_train2.values
X_train4 = []
for str in X_train3:
    str = str.replace("[", "").replace("]", "").replace(" '", "").replace("'", "")
    X_train4.append(str.split(","))
X_train = X_train4.copy()

In [46]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

[['코스피', '내', '린', '장', '종료'],
 ['국내', '투자자', '美中', '수', '하락', '저', '점', '매', '수로', '대응'],
 ['셀트리온', '작년', '영업', '이익', '천억원', '재작년', '보다', '상승'],
 ['클릭', '해외', '증시', '종목', '위드', '코로나', '에어비앤비', '사상', '최대', '매출'],
 ['국제', '유가', '상승', '하락', '다대', '비는', '연', '자금', '으로'],
 ['코스닥', '내린', '개장'],
 ['강', '달러', '에도', '돌아온', '외국인', '삼', '성전', '자', '하이닉스', '집중', '매수'],
 ['뉴욕증시', '유가', '하락', '세', '나스닥', '상승', '마감'],
 ['코스닥', '오른', '개장'],
 ['비트코인', '만원', '대', '우크라이나', '위기', '하락', '세'],
 ['코스피', '오른', '개장'],
 ['코스피', '반', '발매', '수', '유입', '속', '소', '폭', '상승', '출발', '대', '회복', '종합'],
 ['코스피', '내', '린', '장', '종료'],
 ['증시', '아직', '버블', '아냐', '인플레이션', '경', '기', '하강', '우려', '부담'],
 ['상속세', '납부', '위해', '이부진', '삼', '성전', '자', '주식', '담보', '억', '대출'],
 ['코스닥', '내', '린', '장', '종료'],
 ['미국', '장단', '기금', '리차', '벌어져', '침체', '우려', '진정', '될듯'],
 ['메타', '버스', '수익률', '부진', '불구', '하고', '신', '상품', '출시', '잇따라'],
 ['코스피', '하락', '마감', '대로', '밀려'],
 ['대',
  '양',
  '금속',
  '절대',
  '적',
  '저',
  '평가',
  '구간',
  '저',
  '점',


In [51]:
## 재 토큰화  # vocab_szie 1756
tokenizer = Tokenizer(1756, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)




In [52]:
X_train

[[5, 38, 42, 10, 19],
 [209, 261, 1, 66, 6, 57, 48, 78, 958, 959],
 [245, 8, 3, 7, 14, 960, 94, 2],
 [804, 179, 39, 91, 423, 156, 1, 107, 16, 40],
 [692, 180, 2, 6, 1, 805, 25, 37, 21],
 [15, 45, 18],
 [424, 181, 62, 1226, 63, 41, 246, 99, 124, 323, 27],
 [17, 180, 6, 109, 56, 2, 29],
 [15, 13, 18],
 [378, 194, 12, 235, 324, 6, 109],
 [5, 13, 18],
 [5, 112, 469, 66, 425, 65, 136, 159, 2, 28, 12, 84, 11],
 [5, 38, 42, 10, 19],
 [39, 806, 1, 1227, 470, 148, 24, 1, 52, 325],
 [1, 1, 1, 1228, 41, 246, 99, 95, 1, 44, 961],
 [15, 38, 42, 10, 19],
 [167, 807, 1, 1, 1, 962, 52, 471, 1229],
 [262, 263, 210, 100, 1230, 963, 152, 116, 693, 472],
 [5, 6, 29, 211, 247],
 [12, 248, 1231, 1232, 81, 57, 195, 1233, 57, 48, 27, 51, 74, 76, 116, 21],
 [1, 1234, 1235, 532, 3, 46, 2, 11],
 [41, 32, 73, 2, 124, 2, 2, 1, 1, 117, 33],
 [326, 79, 533, 964, 263, 694, 965, 1, 1236, 1, 99, 473],
 [5, 10, 82, 136, 159, 144, 12],
 [212, 305, 4, 3, 7, 14, 8, 22, 9, 6],
 [1, 606, 1, 5, 97, 213, 6],
 [55, 39, 1237, 23

In [53]:
# 리뷰길이 25으로 패딩 , max_len = 25 확인 
X_train = pad_sequences(X_train, maxlen=25)

X_train

array([[  0,   0,   0, ...,  42,  10,  19],
       [  0,   0,   0, ...,  78, 958, 959],
       [  0,   0,   0, ..., 960,  94,   2],
       ...,
       [  0,   0,   0, ...,   5, 131,   1],
       [  0,   0,   0, ..., 107,  16, 383],
       [  0,   0,   0, ...,  90,  35, 125]])

In [47]:
# 모델로드
loaded_model = load_model('test01_model.h5')

In [48]:
## 리뷰예측하기
# 형태소 처리 라이브러리 정의 
okt =  Okt()
# 불용어 : 추가시 리스트에 추가
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

def sentiment_predict(new_sentence):
  new_sentence = new_sentence.replace("↑", "상승").replace("↓", "하락")
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence)
  new_sentence = [word for word in new_sentence if not word in stopwords]
  encoded = tokenizer.texts_to_sequences([new_sentence])
  pad_new = pad_sequences(encoded, maxlen = 25)

  score = float(loaded_model.predict(pad_new))
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))

In [49]:
sentiment_predict('루나코인 -99%↓ 투자자들 패닉')

67.42% 확률로 긍정 리뷰입니다.


In [40]:
sentiment_predict('유럽증시 전쟁연기 소식 15% ↑')

98.35% 확률로 긍정 리뷰입니다.


In [41]:
sentiment_predict('2호 회사 영업실적 최대 기대감 솔솔')

97.56% 확률로 긍정 리뷰입니다.


In [42]:
sentiment_predict('2호 회사 신규 상품 출시')

58.53% 확률로 긍정 리뷰입니다.
